In [1]:
#import the required Python packages
import pandas as pd
import numpy as np

In [2]:
#Note: You might need to add a column for tds level:

scenario = 'phase2_Des_20190609'
df=pd.read_csv('phase2_waterdemand_20190607.csv') 

In [3]:
tds=pd.read_excel('phase2_tds.xlsx')
tds.set_index('ref',inplace=True)

In [4]:
#Set the default values
for i in range (1,13):
    df['osmotic_pressure'.format(i)] = 0
    df['Edesal_GWh_{}'.format(i)] = 0
df['TDS'] = df['Province'].map(tds['TDS'])

In [5]:
#Define inputs:

osm_coeff = 0.95 #osmotic coefficient =1 
molar_mass = 58440 # molar mass of NaCl = 58.44 mg/mol
dissociation = 2 #NaCL dissociation is to 2 ions Na and Cl
concentration = df['TDS'] 
gas_const = 0.083145  #universal gas constant (R) = 0.083145
wtemp = 25 + 273 #water temperature in Kalven (Tc+273): 25 degree is an assumption
threshold = 2000 #Min TDS content for desalination
#solute_concentration = dissociation * concentration / molar_mass

#osmotic_pressure = osm_coeff * solute_concentration * gas_const * temp

#if (df['tds'] > 2000):
#    return df['osmotic_pressure'] = osm_coeff * (dissociation *df['tds']/molar_mass) *  gas_const * wtemp
#else:
#    return df['osmotic_pressure'] = 0

df['osmotic_pressure'] = osm_coeff * (dissociation *df.TDS[df['TDS']>threshold]/molar_mass) *  gas_const * wtemp

df['osmotic_pressure'] = df['osmotic_pressure'].fillna(0)

#df.tds[df['tds']>2000]: This set the criteria that for tds>2000 desalination required

In [6]:
efficiency = 0.85 #overall efficiency
Pf = 55 #feed pressure in bar
Pp = 10 #permeate pressure in bar
Rm = 0.9*(10**14) #mebrane resistance
A = 37 #area m2
N = 15 #number of envelopes
Q = 1
u = 0.00096 #viscosity of water (Ns/m2)

SEC_f = 0.194 #membrane filtration resistance
SEC_R = 0.036 #friction losses, retentate
SEC_p = 0.0016 #friction losses, permeate
SEC_cp = 0.005 #concentration polarization
SEC_inef = 0.068 #pump & ERD inefficiency

In [7]:
#Energy intensity in Kwh/m3

df['Einten_KWh/m3'] = df.loc[df['osmotic_pressure']!=0,'osmotic_pressure']/36 + SEC_f + SEC_R + SEC_p + SEC_cp + SEC_inef
df['Einten_KWh/m3'].fillna(0,inplace=True) 

In [8]:
for i in range(1,13):
    Edesal_GWh = 'Edesal_GWh_{}'.format(i)
    SSWD = 'SSWD_{}'.format(i)
    
    df[Edesal_GWh] = (df['Einten_KWh/m3'] * df[SSWD]) /1000000

In [9]:
#Filter data by provenence: 
df2=df.groupby('NAME_1', as_index=True).sum()
sswd = df.filter(like='sswd_').sum(axis=1)
Edesal = df.filter(like='Edesal_GWh_').sum(axis=1)

# final = df2.filter(['area_m2','sswd_','osmotic_pressure','Edesal_GWh_','country'])
df2.filter(like='Edesal_GWh_').sum(axis=1)

NAME_1
Adrar            0.000000
Beni Walid       0.000000
Biskra          26.756722
Djelfa           0.000000
El Oued        369.598234
Gabes           12.741772
Ghardaia         0.000000
Illizi          10.698618
Jufrah           0.000000
Kebili         142.282078
Khenchela        5.462985
Laghouat         0.000000
Musrata         88.545182
Nalut           27.037841
Ouargla        195.793551
Tamanrasset      0.000000
Tataouine        8.447867
Tebessa         22.329139
Tozeur          46.032087
dtype: float64

In [10]:
df['Einten_KWh/m3'].mean()

0.200806121649925

In [11]:
##Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter(scenario + '.xlsx', engine='xlsxwriter')
#writer.book.use_zip64()

## Convert the dataframe to an XlsxWriter Excel object.
#df.to_excel(writer, sheet_name='All')
#df2.to_excel(writer, sheet_name='Prov')
##sswd.to_excel(writer, sheet_name='SSWD in m3')
##Edesal.to_excel(writer, sheet_name='Edesal in KWh')

## Close the Pandas Excel writer and output the Excel file.
#writer.save()

In [12]:
df.to_csv(scenario + '.csv', index=False)